In [1]:
from platform import python_version
print(python_version())

3.8.7


In [2]:
#https://crime-data-explorer.app.cloud.gov/explorer/state/georgia/crime

In [3]:
import pandas as pd
import numpy as np, sys, time, pdb, matplotlib.pyplot as plt
from pymongo import MongoClient

In [4]:
pd.set_option('display.max_rows', 100)# None

In [5]:
data_2020 = pd.read_csv('./flask/static/data/COBRA-2020.csv')
data_2021 = pd.read_csv('./flask/static/data/COBRA-2021.csv')
data_2009_2019 = pd.read_csv('./flask/static/data/COBRA-2009-2019.csv', low_memory=False)

In [6]:
data_2020_list = data_2020.columns.values.tolist()
data_2021_list = data_2021.columns.values.tolist()
data_2009_2019_list = data_2009_2019.columns.values.tolist()

In [7]:
print(len(data_2020_list))
data_2020.head(1)

20


,offense_id,rpt_date,occur_date,occur_time,poss_date,poss_time,beat,apt_office_prefix,apt_office_num,location,MinOfucr,dispo_code,Shift,loc_type,UC2_Literal,ibr_code,neighborhood,npu,long,lat
0,203341346,12/31/2020,11/29/2020,19:21,11/29/2020,23:50,405,NaN,NaN,3050 M.L.K. JR DR SW,NaN,NaN,NaN,NaN,AGG ASSAULT,13A,Harland Terrace,I,-84.486163,33.75106


In [8]:
print(len(data_2009_2019_list))
data_2009_2019.tail(10)

19


,Report Number,Report Date,Occur Date,Occur Time,Possible Date,Possible Time,Beat,Apartment Office Prefix,Apartment Number,Location,Shift Occurence,Location Type,UCR Literal,UCR #,IBR Code,Neighborhood,NPU,Latitude,Longitude
342904,193651605,2019-12-31,2019-12-31,1615,2019-12-31,1745.0,507.0,NaN,NaN,180 WALKER ST SW,Evening Watch,18,LARCENY-FROM VEHICLE,640,2305,Castleberry Hill,M,33.74964,-84.40132
342905,193651845,2019-12-31,2019-12-31,1855,2019-12-31,1915.0,602.0,NaN,NaN,99 KROG ST NE,Morning Watch,18,LARCENY-FROM VEHICLE,640,2305,Inman Park,N,33.75667,-84.36390
342906,193652247,2019-12-31,2019-12-31,2130,2019-12-31,2132.0,503.0,NaN,NaN,992 PEACHTREE ST NE,Evening Watch,13,LARCENY-FROM VEHICLE,640,2305,Midtown,E,33.78192,-84.38410
342907,193652287,2019-12-31,2019-12-31,2000,2019-12-31,2101.0,501.0,NaN,NaN,1371 MARKET ST NW,Evening Watch,18,LARCENY-FROM VEHICLE,640,2305,Atlantic Station,E,33.79210,-84.39466
342908,193642145,2019-12-31,2019-12-30,2001,2019-12-30,2157.0,501.0,NaN,NaN,1085 HOWELL MILL RD NW,Evening Watch,18,LARCENY-FROM VEHICLE,640,2305,Home Park,E,33.78455,-84.41142
342909,193652089,2019-12-31,2019-12-31,2030,2019-12-31,2034.0,105.0,NaN,NaN,1385 SHARON ST NW,Morning Watch,20,AGG ASSAULT,420,1315K,Hunter Hills,K,33.75486,-84.43287
342910,193650336,2019-12-31,2019-12-31,0432,2019-12-31,432.0,206.0,NaN,NaN,262 PHARR RD NE,Morning Watch,NaN,AGG ASSAULT,410,1314,Buckhead Village,B,33.83732,-84.37860
342911,193650603,2019-12-31,2019-12-31,0920,2019-12-31,940.0,404.0,NaN,NaN,689 CASCADE AVE SW,Day Watch,18,AGG ASSAULT,410,1314,Westview,T,33.73636,-84.43680
342912,193651760,2019-12-31,2019-12-31,1853,2019-12-31,1900.0,404.0,NaN,NaN,763 CASCADE AVE SW,Evening Watch,26,AGG ASSAULT,410,1314,Westview,T,33.73483,-84.43750
342913,193652591,2019-12-31,2019-12-31,2045,2019-12-31,2055.0,406.0,NaN,NaN,237 PEYTON PL SW,Evening Watch,20,AGG ASSAULT,410,1314,Cascade Avenue/Road,S,33.72518,-84.45013


In [9]:
#Since the 2009-2019 dataset is the largest, everything will try and have the same columns as that dataset
data_2020.rename(columns = {
 'offense_id' : 'Report Number',
 'rpt_date' : 'Report Date',
 'occur_date' : 'Occur Date',
 'occur_time' : 'Occur Time',
 'poss_date' : 'Possible Date',
 'poss_time' : 'Possible Time',
 'beat' : 'Beat',
 'apt_office_prefix' : 'Apartment Office Prefix',
 'apt_office_num' : 'Apartment Number',
 'location' : 'Location',
 #'MinOfucr' : ,
 #'dispo_code' : ,
 'Shift' : 'Shift Occurence',
 'loc_type' : 'Location Type',
 'UC2_Literal': 'UCR Literal',
 'ibr_code' : 'IBR Code',
 'neighborhood' : 'Neighborhood',
 'npu' : 'NPU',
 'long' : 'Longitude',
 'lat' : 'Latitude'
}, inplace = True)
#Drop columns that do not seem to show up in 2009-2019 dataset
data_2020 = data_2020.drop(columns=['MinOfucr','dispo_code'])
data_2020.head(1)

,Report Number,Report Date,Occur Date,Occur Time,Possible Date,Possible Time,Beat,Apartment Office Prefix,Apartment Number,Location,Shift Occurence,Location Type,UCR Literal,IBR Code,Neighborhood,NPU,Longitude,Latitude
0,203341346,12/31/2020,11/29/2020,19:21,11/29/2020,23:50,405,NaN,NaN,3050 M.L.K. JR DR SW,NaN,NaN,AGG ASSAULT,13A,Harland Terrace,I,-84.486163,33.75106


In [10]:
list1 = data_2009_2019.columns.values.tolist()
list2 = data_2020.columns.values.tolist()

list_difference = [item for item in list1 if item not in list2]
print(list_difference)

['UCR #']


In [11]:
#It can be seen that the 'UCR #' shows up only in the 2009-2019 data and realistically serves no purpose because its a code only a cop would udnerstand
#So it should be droped to simplify
data_2009_2019 = data_2009_2019.drop(columns=['UCR #'])

In [12]:
list1 = data_2009_2019.columns.values.tolist()
list2 = data_2020.columns.values.tolist()

list_difference = [item for item in list1 if item not in list2]
print(list_difference)
#There should now be no differences as at all

[]


In [13]:
#Concat the dataframes together
data = pd.concat([data_2009_2019, data_2020])

In [14]:
data_2009_2019['Occur Date'].sort_values();

In [15]:
data['Report Date'] = pd.to_datetime(data['Report Date'])

In [16]:
data.to_csv('data.csv')
data.reset_index(drop=True, inplace=True)
#data.to_json('data.json')

# Decrease Data by Reducing Number of Neighborhods
It has become apparent that there is too mach data and our relatively weak computers are not up to the challenge give the time.

It was decided to just look at 5 major "Neighborhood" of interest:

buckhead, midtown, brookhaven, downtown, old fourth ward

In [17]:
df = pd.read_csv("data.csv")
df_reduced = df

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (4,6,7,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
df_reduced["Neighborhood"].unique();
#Used to show all rows for error checking
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 100)

In [19]:
df_buckhead = df_reduced[(df_reduced["Neighborhood"] == "Buckhead Forest") |(df_reduced["Neighborhood"] == "Buckhead Village")
           |(df_reduced["Neighborhood"] == "North Buckhead") |(df_reduced["Neighborhood"] == "Buckhead Heights")
          ]
df_buckhead
df_buckhead['Neighborhood']

df_buckhead = df_buckhead.assign(Neighborhood = 'Buckhead' )
df_buckhead

df_other_hoods  = df_reduced[(df_reduced["Neighborhood"] == "Midtown") | (df_reduced["Neighborhood"] == "Brookhaven")
          |(df_reduced["Neighborhood"] == "Downtown") | (df_reduced["Neighborhood"] == "Old Fourth Ward")]
df_other_hoods

df_5hoods = pd.concat([df_buckhead,df_other_hoods], ignore_index=True, sort=False)
df_5hoods

df_5hoods = df_5hoods.drop(columns=['Unnamed: 0'])
df_5hoods.to_csv('data_5hoods.csv', index= False)

# Seperate out the Date and Time Columns

Need to make the columns a datetime formate and then seperate at useful information such as year, month, day, day of the week and make sure they are all in some sort of numerical formatting that would make sense in MachinLearning

In [185]:
df = pd.read_csv("data_5hoods.csv")

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (7,8,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [186]:
df['Occur Date'] = pd.to_datetime(df['Occur Date'], errors='coerce')

In [187]:
df['Occur Year'] = pd.DatetimeIndex(df['Occur Date']).year
df['Occur Month'] = pd.DatetimeIndex(df['Occur Date']).month
df['Occur Day'] = pd.DatetimeIndex(df['Occur Date']).day
df['Occur DayofWeek'] = pd.DatetimeIndex(df['Occur Date']).dayofweek#Already sets to a numerical for onehotencoding later
df = df.dropna(subset=['Occur Year'], how='all')
df = df[~(df['Occur Year'] <= 2008)]
df.sort_values(by=['Occur Year']).head(100);

In [188]:
#df['Occur Time'] = pd.to_datetime(df['Occur Time'], errors='coerce')
df = df[~df['Occur Time'].str.contains(":")]
df = df[~df['Location Type'].str.contains(r"[a-zA-Z]",na=False)]#Drop the weird nonnumeric character that is in a few

In [189]:
df["Location Type"] = pd.to_numeric(df["Location Type"])
df = df.dropna(subset=['Location Type'])

In [190]:
df.to_csv('data_5hoods_datetime.csv', index= False)

# Machine Learnings

## Read the CSV and Perform Basic Data Cleaning

In [ ]:
df = pd.read_csv("data.csv")
# Drop the null columns where all values are null
#df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df[["Latitude","Longitude","Neighborhood"]]
df = df.dropna()

## Create a Train Test Split

Use `Neighborhood` for the y values
This can be an idiot check to see if lat an lon can predict neighborhood

In [ ]:
from sklearn.model_selection import train_test_split
y = df["Neighborhood"]
X = df.drop(columns=["Neighborhood"])

#For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)#Compute the minimum and maximum to be used for later scaling.

X_train_scaled = X_scaler.transform(X_train)#Scale features of X_train according to feature_range.
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
y_train.nunique()

# Train the Model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(max_iter=200)
model.fit(X_train_scaled, y_train)# Fit the LR model

In [ ]:
df

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

In [ ]:
model.predict()

In [ ]:
X_scaler.transform(X_train)[0]

In [ ]:
model.predict(X_train_scaled)

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
# Cfloat, default=1.0 Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
# penalty{‘l1’, ‘l2’, ‘elasticnet’, ‘none’}, default=’l2’
param_grid = {'C': [1, 3,5,7,9],
              'penalty': ["l1", "l2", "none"]}
Model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(Model, param_grid, verbose=3)
grid

In [ ]:
idk = grid.fit(X_train_scaled, y_train)
idk

# Machine Learnings

In [232]:
import pandas as pd
df = pd.read_csv("data_5hoods_datetime.csv")

In [233]:
pd.options.display.max_columns=50

In [234]:
#Select columns of interst
df = df[["Beat",
    "Occur Time",
    "Neighborhood",
    "Location Type",
    "Latitude",
    "Longitude",
    "Occur Year",
    "Occur Month",
    "Occur Day",
    "Occur DayofWeek",
    "UCR Literal"]]
#df.dropna(subset=['Location Type', 'Occur Time'])
df.dropna(subset=['Beat'], inplace=True)

In [235]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 
y = df["UCR Literal"]
X = df.drop(columns=["UCR Literal"])

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() 
X['Neighborhood']= le.fit_transform(X['Neighborhood']) 

In [236]:
#For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [237]:
X_train.sort_values(['Beat'])

,Beat,Occur Time,Neighborhood,Location Type,Latitude,Longitude,Occur Year,Occur Month,Occur Day,Occur DayofWeek
58111,102.0,2330,2,28.0,33.76951,-84.38527,2019.0,2.0,4.0,0.0
60324,201.0,445,2,26.0,33.75657,-84.38034,2019.0,8.0,26.0,0.0
8624,206.0,1630,1,99.0,33.84008,-84.37231,2018.0,12.0,22.0,5.0
6092,206.0,1530,1,18.0,33.83790,-84.37852,2016.0,3.0,15.0,1.0
9090,206.0,1545,1,8.0,33.83852,-84.38007,2019.0,7.0,25.0,3.0
...,...,...,...,...,...,...,...,...,...,...
38022,614.0,2150,3,18.0,33.77909,-84.36717,2014.0,4.0,13.0,6.0
50904,614.0,2130,3,7.0,33.77604,-84.37739,2017.0,4.0,17.0,0.0
45026,614.0,1830,3,13.0,33.77749,-84.37164,2015.0,9.0,25.0,4.0
18980,614.0,830,3,13.0,33.77873,-84.37368,2010.0,7.0,25.0,6.0


In [238]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)#Compute the minimum and maximum to be used for later scaling.

X_train_scaled = X_scaler.transform(X_train)#Scale features of X_train according to feature_range.
X_test_scaled = X_scaler.transform(X_test)

In [239]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
# Cfloat, default=1.0 Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
# penalty{‘l1’, ‘l2’, ‘elasticnet’, ‘none’}, default=’l2’
param_grid = {'C': [6,7,11],
              'penalty': ["l1", "l2", "none"]}
Model = LogisticRegression(solver='liblinear')
grid = GridSearchCV(Model, param_grid, verbose=1)
grid

GridSearchCV(estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': [6, 7, 11], 'penalty': ['l1', 'l2', 'none']},
             verbose=3)

In [240]:
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END ................................C=6, penalty=l1; total time=  10.6s
[CV 2/5] END ................................C=6, penalty=l1; total time=   9.1s
[CV 3/5] END ................................C=6, penalty=l1; total time=   9.4s
[CV 4/5] END ................................C=6, penalty=l1; total time=   8.7s
[CV 5/5] END ................................C=6, penalty=l1; total time=  10.2s
[CV 1/5] END ................................C=6, penalty=l2; total time=   0.7s
[CV 2/5] END ................................C=6, penalty=l2; total time=   0.8s
[CV 3/5] END ................................C=6, penalty=l2; total time=   0.7s
[CV 4/5] END ................................C=6, penalty=l2; total time=   0.7s
[CV 5/5] END ................................C=6, penalty=l2; total time=   0.8s
[CV 1/5] END ..............................C=6, penalty=none; total time=   0.0s
[CV 2/5] END ..............................C=6, p

/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 454, in _check_solver
    raise ValueError(
ValueError: penalty='none' is not supported for the liblinear solver

  warnings.warn("Estimator fit failed. The score on this train-test"
/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-te

[CV 1/5] END ................................C=7, penalty=l1; total time=  10.3s
[CV 2/5] END ................................C=7, penalty=l1; total time=   9.1s
[CV 3/5] END ................................C=7, penalty=l1; total time=   9.6s
[CV 4/5] END ................................C=7, penalty=l1; total time=   9.4s
[CV 5/5] END ................................C=7, penalty=l1; total time=  10.8s
[CV 1/5] END ................................C=7, penalty=l2; total time=   0.7s
[CV 2/5] END ................................C=7, penalty=l2; total time=   0.7s
[CV 3/5] END ................................C=7, penalty=l2; total time=   0.7s
[CV 4/5] END ................................C=7, penalty=l2; total time=   0.7s
[CV 5/5] END ................................C=7, penalty=l2; total time=   0.7s
[CV 1/5] END ..............................C=7, penalty=none; total time=   0.0s
[CV 2/5] END ..............................C=7, penalty=none; total time=   0.0s
[CV 3/5] END ...............

/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 454, in _check_solver
    raise ValueError(
ValueError: penalty='none' is not supported for the liblinear solver

  warnings.warn("Estimator fit failed. The score on this train-test"
/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-te

[CV 1/5] END ...............................C=11, penalty=l1; total time=  10.0s
[CV 2/5] END ...............................C=11, penalty=l1; total time=  10.2s
[CV 3/5] END ...............................C=11, penalty=l1; total time=   9.8s
[CV 4/5] END ...............................C=11, penalty=l1; total time=  10.1s
[CV 5/5] END ...............................C=11, penalty=l1; total time=   9.9s
[CV 1/5] END ...............................C=11, penalty=l2; total time=   0.8s
[CV 2/5] END ...............................C=11, penalty=l2; total time=   0.7s
[CV 3/5] END ...............................C=11, penalty=l2; total time=   0.7s
[CV 4/5] END ...............................C=11, penalty=l2; total time=   0.7s
[CV 5/5] END ...............................C=11, penalty=l2; total time=   0.7s
[CV 1/5] END .............................C=11, penalty=none; total time=   0.0s
[CV 2/5] END .............................C=11, penalty=none; total time=   0.0s
[CV 3/5] END ...............

/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 454, in _check_solver
    raise ValueError(
ValueError: penalty='none' is not supported for the liblinear solver

  warnings.warn("Estimator fit failed. The score on this train-test"
/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-te

GridSearchCV(estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': [6, 7, 11], 'penalty': ['l1', 'l2', 'none']},
             verbose=3)

In [241]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 6, 'penalty': 'l1'}
0.4600005402464295


# Save the Model

In [ ]:
# save fitted model to file
# In the specific case of scikit-learn, it may be better to use joblib’s replacement of
# pickle (dump & load), which is more efficient on objects that carry large numpy arrays
# internally as is often the case for fitted scikit-learn estimators, but can only pickle
# to the disk and not to a string:

import joblib
filename = 'five_hoods_LR.sav'
joblib.dump(grid, filename)

## Test

In [ ]:
Model = joblib.load("five_hoods_LR.sav")
Model.predict(X_test_scaled)